In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_42277/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/21 03:51:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/21 03:51:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/03/21 03:51:53 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
import pandas as pd

# Sample data for Sessions table
sessions_data = {
    'user_id': [101, 101, 102, 102, 101, 104, 103],
    'session_start': ['2023-11-06 13:53:42', '2023-11-22 16:45:21', '2023-11-16 13:23:09', '2023-11-17 13:23:09', '2023-11-20 07:16:06', '2023-11-27 03:10:49', '2023-11-27 03:10:49'],
    'session_end': ['2023-11-06 14:05:42', '2023-11-22 20:39:21', '2023-11-16 16:10:09', '2023-11-17 16:10:09', '2023-11-20 08:33:06', '2023-11-27 03:30:49', '2023-11-27 03:30:49'],
    'session_id': [375, 594, 777, 778, 315, 797, 798],
    'session_type': ['Viewer', 'Streamer', 'Streamer', 'Streamer', 'Streamer', 'Viewer', 'Streamer']
}

# Create DataFrame for Sessions
sessions_df = pd.DataFrame(sessions_data)

# Display the DataFrame
print("Sessions table:")
print(sessions_df)



df_person = spark.createDataFrame(sessions_df)
df_person.createOrReplaceTempView("Sessions")



Sessions table:
   user_id        session_start          session_end  session_id session_type
0      101  2023-11-06 13:53:42  2023-11-06 14:05:42         375       Viewer
1      101  2023-11-22 16:45:21  2023-11-22 20:39:21         594     Streamer
2      102  2023-11-16 13:23:09  2023-11-16 16:10:09         777     Streamer
3      102  2023-11-17 13:23:09  2023-11-17 16:10:09         778     Streamer
4      101  2023-11-20 07:16:06  2023-11-20 08:33:06         315     Streamer
5      104  2023-11-27 03:10:49  2023-11-27 03:30:49         797       Viewer
6      103  2023-11-27 03:10:49  2023-11-27 03:30:49         798     Streamer


In [19]:
query = """
    with get_rank as (
        select
            *,
            row_number() over(partition by user_id order by session_start asc) as rk,
            count(*) over(partition by user_id) as count_session
        from Sessions s 
    )
    
    select 
        distinct 
        u.user_id,
        count(*) as session_count
    from (
        select
            distinct user_id
        from get_rank where rk = 1 and session_type = 'Viewer' and count_session >= 2
    ) u left join Sessions s on u.user_id = s.user_id
    where s.session_type = 'Streamer'
    group by u.user_id
"""

In [20]:
spark.sql(query).show()

+-------+-------------+
|user_id|session_count|
+-------+-------------+
|    101|            2|
+-------+-------------+

